# Environment
Kernel: conda env:bdm

In [1]:
pip list

Package           Version
----------------- -----------
appnope           0.1.2
asttokens         2.0.5
backcall          0.2.0
certifi           2022.5.18.1
debugpy           1.5.1
decorator         5.1.1
entrypoints       0.4
executing         0.8.3
future            0.18.2
hdfs3             0.3.1
ipykernel         6.9.1
ipython           8.3.0
jedi              0.18.1
jupyter-client    7.2.2
jupyter-core      4.10.0
kafka-python      2.0.2
matplotlib-inline 0.1.2
nest-asyncio      1.5.5
numpy             1.22.2
pandas            1.4.1
parso             0.8.3
pexpect           4.8.0
pickleshare       0.7.5
pip               21.2.4
prompt-toolkit    3.0.20
ptyprocess        0.7.0
pure-eval         0.2.2
py4j              0.10.9.3
pyarrow           5.0.0
Pygments          2.11.2
pymonetdb         1.6.1
pymongo           3.12.2
pyspark           3.2.1
python-dateutil   2.8.2
pytz              2022.1
pyzmq             22.3.0
setuptools        61.2.0
six               1.16.0
stack-data   

# Machine Learning with Spark

- https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [2]:
#from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
#from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegressionSummary
from pyspark.ml.regression import LinearRegressionTrainingSummary
from pyspark.ml import Pipeline
import os

#Cannot be imported -- do I need to install something?
#from utils.properties_parser import parse_properties

In [3]:
spark = SparkSession.builder.master("local[*]").appName("ml-test").config('spark.driver.extraClassPath',
                './drivers/monetdb-jdbc-3.2.jre8.jar').getOrCreate()

22/06/17 20:17:53 WARN Utils: Your hostname, Kathryns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.37 instead (on interface en0)
22/06/17 20:17:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 20:17:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Reading the idealista table from the formatted zone into a Spark dataframe
df = spark.read.format("jdbc") \
    .option("url", "jdbc:monetdb://dodrio.fib.upc.es:50000/mydb") \
    .option("dbtable", 'idealista') \
    .option("user", 'monetdb') \
    .option("password", 'monetdb') \
    .option("batchsize", 10000) \
    .load()

In [5]:
# verify columns and datatypes of dataframe
print(df.dtypes)

[('district', 'string'), ('neighborhood', 'string'), ('propertyCode', 'string'), ('thumbnail', 'string'), ('externalReference', 'string'), ('numPhotos', 'bigint'), ('floor', 'string'), ('price', 'double'), ('propertyType', 'string'), ('operation', 'string'), ('size', 'double'), ('exterior', 'boolean'), ('rooms', 'bigint'), ('bathrooms', 'bigint'), ('address', 'string'), ('province', 'string'), ('municipality', 'string'), ('country', 'string'), ('latitude', 'double'), ('longitude', 'double'), ('showAddress', 'boolean'), ('url', 'string'), ('distance', 'string'), ('hasVideo', 'boolean'), ('status', 'string'), ('newDevelopment', 'boolean'), ('hasLift', 'boolean'), ('priceByArea', 'double'), ('detailedType', 'string'), ('suggestedTexts', 'string'), ('hasPlan', 'boolean'), ('has3DTour', 'boolean'), ('has360', 'boolean'), ('hasStaging', 'boolean'), ('topNewDevelopment', 'boolean'), ('_process_time', 'timestamp'), ('_input_file_name', 'string'), ('district_n_reconciled', 'string'), ('district

## Pre-processing data for ML model training

Reference for OHE Steps: https://towardsdev.com/how-to-write-pyspark-one-hot-encoding-results-to-an-interpretable-csv-file-626ecb973962

### StringIndexer:

- Spark's StringIndexer is a required pre-processing step to convert the categorical data of neighorhood_id from a string to a number in order for the Spark OneHotEncoder function to work.

- Using the option to ignore all the missing values of neighborhood_id in the idealista dataframe with the StringIndexer parameter `handleInvalid='skip'` provides better regression results than keeping the missing values as their own category.

- The alternative option, `handleInvalid='keep'` in the StringIndexer will categorize the missing values in their own category together, which wouldn't make much sense. Properties in Sant Cugat would be grouped with properties in L'Hospitalet because they are both missing Barcelona neighborhood ids.

Reference for StringIndexer: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html#pyspark.ml.feature.StringIndexer.handleInvalid

In [6]:
# numeric indexing for the strings (indexing starts from 0)

# rows with missing neighborhood_ids will be filtered out with handleInvalid='skip'
indexer = StringIndexer(inputCol="neighborhood_id", outputCol="neighborhood_id_index", handleInvalid='skip')

# fit the indexer model and use it to transform the strings into numeric indices
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)

### OneHotEncoder:
- One Hot Encoding is required in order to perform a linear regression to estimate the price based on the neighborhood.

In [7]:
# one-hot-encoding the numeric indices
ohe = OneHotEncoder(inputCol="neighborhood_id_index", outputCol="neighborhoodOHEVector")

# fit the ohe model and use it to transform the numeric indices into ohe vectors
ohe_model = ohe.fit(df)
df = ohe_model.transform(df)

In [8]:
# create dataframe for machine learning
ml_df = df.select(['neighborhoodOHEVector', 'price'])
# preview 3 rows of data
ml_df.show(3)

+---------------------+---------+
|neighborhoodOHEVector|    price|
+---------------------+---------+
|       (53,[0],[1.0])| 680000.0|
|       (53,[0],[1.0])|1100000.0|
|      (53,[17],[1.0])| 358000.0|
+---------------------+---------+
only showing top 3 rows



### Create train and test sets for model validation

In [9]:
# split data into training and test sets
splits = ml_df.randomSplit([0.7, 0.3], seed=24)
train_df = splits[0]
test_df = splits[1]

### Create linear regression model to predict price based on neighborhood

In [10]:
# create linear regression model
lr = LinearRegression(featuresCol='neighborhoodOHEVector', labelCol='price', maxIter=10, regParam=0.1)
# fit the linear regression model to the training data
lr_model = lr.fit(train_df)

22/06/17 20:18:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/17 20:18:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/06/17 20:18:09 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


The coefficients can be interpreted as the variation of property prices in euros per neighborhood, because only one variable will have the value of 1 per sample to represent the neighborhood in which the property is located. Large positive coefficients are for expensive neighborhoods and negative coefficeints are for neighborhoods with lower property values.

In [11]:
# Check coefficients of variables and the model intercept
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [774355.505058845,736292.7401970942,12305.69217641256,189553.5119624583,264488.518061847,1303205.4297686112,88088.47521745606,-26397.696647085464,-73324.51404720619,365706.75560065225,-69208.96266737235,194292.83664842194,-45040.69668952972,67759.74584644934,1441425.646377325,317489.888583991,1607807.9097785098,37039.28914200649,423310.74459833687,856127.9043658309,357665.65618308546,962161.0998884225,-41650.30943947419,232482.8138624805,425841.19469307456,121628.70877362823,12802.635633079322,133759.14194420117,107654.97860453339,-9675.622368122962,-94397.97180069648,27409.155503466132,-76070.24304020684,634.1589247406345,-134518.6015587498,-15640.838991094037,-27240.83750488134,65759.15058463665,49759.15263237194,-80240.83071827103,-153765.8212985952,-188340.81687473896,-81907.49716734886,-141740.82283884878,-206907.48115322838,119759.14365713298,-154740.82116978796,99759.14622040612,-122240.8253303612,39659.15392304154,0.0,-161240.82032888086,-11240.83955342856]
Interc

An r2 score of 0.364 means 36.4% of the variance in the price is explained by our model, which is only based on neighborhoods.

(Using the rows with missing neighborhood id values as their own category in the regression model had a worse r2 score of 0.29)

The mean absolute error represents the average price difference between the model's prediction and the true price in euros.

In [12]:
# save training summary from the regression model to get training metrics
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("Mean Absolute Error: %f" % trainingSummary.meanAbsoluteError)

RMSE: 622125.230828
r2: 0.364000
Mean Absolute Error: 321297.448040


In [13]:
# check summary statistics of the training dataset
train_df.describe().show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|             5058|
|   mean|710042.0943060499|
| stddev|780175.1904303073|
|    min|          39000.0|
|    max|            1.2E7|
+-------+-----------------+



### Make predictions and evaluate model performance

In [14]:
##### save predictions from model using test data
lr_predictions = lr_model.transform(test_df)
# preview 5 rows of predictions
lr_predictions.select("prediction","price",'neighborhoodOHEVector').show(5)

+-----------------+--------+---------------------+
|       prediction|   price|neighborhoodOHEVector|
+-----------------+--------+---------------------+
|1094596.346052941|230000.0|       (53,[0],[1.0])|
|1094596.346052941|250000.0|       (53,[0],[1.0])|
|1094596.346052941|280000.0|       (53,[0],[1.0])|
|1094596.346052941|320000.0|       (53,[0],[1.0])|
|1094596.346052941|320000.0|       (53,[0],[1.0])|
+-----------------+--------+---------------------+
only showing top 5 rows



The evaluation criteria on the test set are similar to the results achieved on the training data, which means the model is not overfitting and the training set is a good sample of the overall population.

In [15]:
# saving the evaluation results to a spark linear regression summary
testSummary = lr_model.evaluate(test_df)
print("RMSE: %f" % testSummary.rootMeanSquaredError)
print("r2: %f" % testSummary.r2)
print("Mean Absolute Error: %f" % testSummary.meanAbsoluteError)

RMSE: 607212.762940
r2: 0.358802
Mean Absolute Error: 316830.047112


In [16]:
# summary statistics of the testing dataset
test_df.describe().show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|             2164|
|   mean|707250.5670055453|
| stddev|758481.9907412607|
|    min|          34000.0|
|    max|            1.2E7|
+-------+-----------------+



# Creating ML Pipeline

First I created the previous model step by step, and now I use a pipeline to create a model and doublecheck that the models and evaluation criteria are similar for correctness. The pipeline is the preferred way to implement the model since it includes the pre-processing steps that will be necessary on the streaming data.

In [17]:
# create dataframe for machine learning
ml2_df = df.select(['neighborhood_id', 'price'])
# preview 3 rows of machine learning dataframe
ml2_df.show(3)

+---------------+---------+
|neighborhood_id|    price|
+---------------+---------+
|       Q1904302| 680000.0|
|       Q1904302|1100000.0|
|       Q1758503| 358000.0|
+---------------+---------+
only showing top 3 rows



In [18]:
# split data into training and test sets
splits2 = ml2_df.randomSplit([0.7, 0.3], seed=24)
training = splits2[0]
testing = splits2[1]

In [19]:
# Configure an ML pipeline, which consists of three stages: StringIndexer, OneHotEncoder, and LinearRegression.
indexer = StringIndexer(inputCol="neighborhood_id", outputCol="neighborhood_id_index", handleInvalid='skip')
ohe = OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="neighborhoodOHEVector")
lr = LinearRegression(featuresCol='neighborhoodOHEVector', labelCol='price', maxIter=10, regParam=0.1)
pipeline = Pipeline(stages=[indexer, ohe, lr])

# Fit the pipeline to training dataset.
model = pipeline.fit(training)

The linear model attributes can be accessed by indexing its stage in the pipeline, which is the last step.

In [20]:
# Check linear regerssion coefficients of variables and the intercept
print("Coefficients: " + str(model.stages[-1].coefficients))
print("Intercept: " + str(model.stages[-1].intercept))

Coefficients: [801551.7984362687,721980.3515125262,182189.82923183558,14993.09625627282,1368051.1830096587,251694.60557201778,87724.60944513443,-67727.06718056908,-32083.048894254705,388446.26833243767,192705.57320146065,-71194.69478207281,64403.74672005009,-42578.19668388537,1535186.1973954593,1514518.5682822552,323989.4839252275,46925.4006723051,837942.650995285,412503.998653938,380486.77763261244,-32290.746326888748,934112.1738481588,200887.30092211097,429523.9182867505,135456.41200154502,4508.754042766958,99830.17059543215,-41396.24016092217,-113619.00881218963,-84630.12358568041,-8064.773781419348,171082.26240465321,-47779.70088485023,-21796.798185626987,-136074.56150007088,-37812.66917258799,34258.75027279512,-88574.56750515067,-148591.22657141698,46758.74868864605,-141741.2274317823,-202991.21967212422,-162991.22473965606,-218574.55102575265,-55241.238387550395,99758.74196799309,-7741.2444076258025,48708.74843798989,-122241.22989301143,-11241.243963759569]
Intercept: 320241.2453

In [21]:
# get linear regression training summary
Summary = model.stages[-1].summary
print("RMSE: %f" % Summary.rootMeanSquaredError)
print("r2: %f" % Summary.r2)
print("Mean Absolute Error: %f" % Summary.meanAbsoluteError)


RMSE: 626954.920842
r2: 0.368122
Mean Absolute Error: 325512.921458


In [22]:
# save the model's predictions of the testing dataset
prediction = model.transform(testing)

In [23]:
# get linear regression testing summary
t_Summary = lr_model.evaluate(test_df)
print("RMSE: %f" % testSummary.rootMeanSquaredError)
print("r2: %f" % testSummary.r2)
print("Mean Absolute Error: %f" % testSummary.meanAbsoluteError)

RMSE: 607212.762940
r2: 0.358802
Mean Absolute Error: 316830.047112


In [25]:
model.save("Fitted-ML-Pipeline")

In [26]:
spark.stop()